In [ ]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
from segmentation import segment_scene_colorful
from utils import show

In [ ]:
calib = np.load('phitz/calib.npz')
K = calib['K']
D = calib['D']

In [ ]:
# segment scene
view = "top"
image_file = "phitz/test/0000.jpg"
image = cv2.imread(image_file)
segmentation, (num_red, num_blue) = segment_scene_colorful(image, view)
print(f"Segmented {num_red} red, {num_blue} blue")
show(segmentation)
# cv2.imwrite("segmentation.jpg", segmentation)